In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Parameters
img_height, img_width = 224, 224
batch_size = 32
epochs = 10
train_dir = '/content/drive/MyDrive/data/train'
validation_dir = '/content/drive/MyDrive/data/test'

# Data preparation using image_dataset_from_directory
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

# Determine the correct number of classes
num_classes = len(train_dataset.class_names)  # Get the number of classes from the dataset



Found 15590 files belonging to 23 classes.
Found 4029 files belonging to 23 classes.


In [ ]:
# Load the pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)


In [ ]:
# Train the model with callbacks
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 2468s 5s/step - accuracy: 0.1552 - loss: 2.9312 - val_accuracy: 0.1770 - val_loss: 2.7113 - learning_rate: 0.0010
Epoch 2/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1104s 2s/step - accuracy: 0.2125 - loss: 2.6386 - val_accuracy: 0.2122 - val_loss: 2.6492 - learning_rate: 0.0010
Epoch 3/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1092s 2s/step - accuracy: 0.2381 - loss: 2.5563 - val_accuracy: 0.2303 - val_loss: 2.5886 - learning_rate: 0.0010
Epoch 4/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1117s 2s/step - accuracy: 0.2555 - loss: 2.4778 - val_accuracy: 0.2254 - val_loss: 2.6091 - learning_rate: 0.0010
Epoch 5/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1155s 2s/step - accuracy: 0.2672 - loss: 2.4167 - val_accuracy: 0.2316 - val_loss: 2.5940 - learning_rate: 0.0010
Epoch 6/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1087s 2s/step - accuracy: 0.2961 - loss: 2.3436 - val_accuracy: 0.2417 - val_loss: 2.5880 - learning_rate: 0.0010
Epoch 7/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 1112s 2s/step - accuracy: 0.3107 - l

In [ ]:
model_path = '/content/drive/MyDrive/demoimg_skinvmax/mobilenetv2_image_classifier.keras'
# Save the model
model.save(model_path)
print("Model training complete and saved in Keras format.")

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/demoimg_skinvmax/mobilenetv2_image_classifier.keras')

Model training complete and saved in Keras format.
